In [1]:
from delta import *
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from dotenv import load_dotenv
import os

load_dotenv()

spark = pyspark.sql.SparkSession.builder.appName("DeltaLake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0,org.apache.hadoop:hadoop-aws:3.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_USER")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_PASSWORD")) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("com.amazonaws.services.s3.enableV4", True) \
    .config("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
    .getOrCreate()

#spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()


21/09/13 19:32:31 WARN Utils: Your hostname, DESKTOP-58VHONS resolves to a loopback address: 127.0.1.1; using 172.18.143.150 instead (on interface eth0)
21/09/13 19:32:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/smikic/.cache/pypoetry/virtualenvs/realestatedataengineering-la76Cphi-py3.8/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/smikic/.ivy2/cache
The jars for the packages stored in: /home/smikic/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11f247ea-29ea-4d38-a70b-eba931652f0b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.7.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 937ms :: artifacts dl 29ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	com.ibm.icu#icu4j;5

In [2]:
print(os.getenv("MINIO_USER"))
print(os.getenv("MINIO_PASSWORD"))

admin
superadmin


In [29]:
import boto3

s3 = boto3.client(
    service_name="s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id=os.getenv("MINIO_USER"),
    aws_secret_access_key=os.getenv("MINIO_PASSWORD")
)

s3Resource = boto3.resource(
    service_name="s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id=os.getenv("MINIO_USER"),
    aws_secret_access_key=os.getenv("MINIO_PASSWORD")
)

if s3Resource.Bucket("apt").creation_date is None:
    s3.create_bucket(Bucket="apt")
    df = spark.read.csv("../data/apt_data.csv")
    df.write.format("delta").save("s3a://apt/delta")

In [30]:
# Bucket needs to be created first

df = spark.read.format("delta").load("s3a://apt/delta")
df.toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7
0,id,title,price,number_of_room,apt_size,district,city,street_address
1,0,2 Zimmer Mietwohnung im Altstadtkern von Bludenz,€ 790,2 Zimmer,"72,8 m",Bludenz,Bludenz,None
2,2,Sonnige 3-Zimmer-Maisonette mit Balkon,"€ 955,52",3 Zimmer,69 m,Bregenz,Wolfurt,None
3,4,Helle 2-Zimmerwohnung in guter Wohnlage,"€ 871,61",2 Zimmer,"52,15 m",Dornbirn,Dornbirn,None
4,6,3-Zimmer Wohnung mit Balkon,"€ 858,36",3 Zimmer,70 m,Feldkirch,Frastanz,None
...,...,...,...,...,...,...,...,...
457,950,3 1/2 -Zi-Panoramawohnung mit 2 Terrassen,€ 1.030,N/A,90 m,Feldkirch,Feldkirch,", Liechtensteinerstraße 59"
458,952,Neue 3-Zimmerwohnung im 1. Obergeschoss Dornbi...,"€ 1.685,92",3 Zimmer,"94,61 m",Dornbirn,Dornbirn,", Bergmannstraße 2"
459,954,Schöne renovierte 4 Zimmerwohnung in Bregenz,€ 1.250,N/A,90 m,Bregenz,Bregenz,", Felchenstraße"
460,956,Büroräumlichkeiten / Top 6,Preis auf Anfrage,N/A,110 m,Bludenz,Bludenz,", Wichnerstraße"


In [32]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---+---+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|_c5|_c6|_c7|
+---+---+---+---+---+---+---+---+
|  0|  1|  1|  1|  1|  1|  7|130|
+---+---+---+---+---+---+---+---+

